In [27]:
%load_ext autoreload
%autoreload 2

import os,sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import util as yu
from util import *
import util_NpiScatteringWilson as yun

yu.setpath('autocorrelation')

ens='milc'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# toy model example

def run(tau,Njk,djks):
    Ncfg_old=Njk
    Ncfg_new=djks[-1]*(Ncfg_old//djks[-1])

    dplt=0.1
    ts=np.arange(10)
    dat=yu.random_autocorrelation(Ncfg_new,len(ts),tau=tau)
    mean,delta,tint,d_tint=yu.tauint(dat)
    
    fig,axs=yu.getFigAxs(1,1,Lrow=4,Lcol=16)
    fig.suptitle(f'tau={tau}; Njk={Ncfg_new}; djks={djks}')
    ax=axs[0,0]
    
    plt_x=ts-dplt; plt_y=mean; plt_yerr=delta
    ax.errorbar(plt_x,plt_y,plt_yerr,color='r',label='err from puwr')
    
    dat_jk=yu.jackknife(dat)
    mean,err=yu.jackme(dat_jk)
    plt_x=ts; plt_y=mean; plt_yerr=err*np.sqrt(2*tau)
    ax.errorbar(plt_x,plt_y,plt_yerr,color='black',label='jk1err*sqrt(2*tau_exact)')
    
    plt_x=ts-dplt*2; plt_y=mean; plt_yerr=err
    ax.errorbar(plt_x,plt_y,plt_yerr,color='g',label='jk1err')
    plt_x=ts-dplt*3; plt_y=mean; plt_yerr=err*np.sqrt(2*tint)
    ax.errorbar(plt_x,plt_y,plt_yerr,color='b',label='jk1err*sqrt(2*[tau_int+d_tau]) ')
    plt_x=ts-dplt*3; plt_y=mean; plt_yerr=err*np.sqrt(2*(tint+d_tint))
    ax.errorbar(plt_x,plt_y,plt_yerr,color='b')
    
    for idjk,djk in enumerate(djks):
        dat_jk=yu.jackknife(dat,d=djk)
        mean,err=yu.jackme(dat_jk)
        plt_x=ts+dplt*(idjk+1); plt_y=mean; plt_yerr=err
        ax.errorbar(plt_x,plt_y,plt_yerr,color='orange',label=f'jkderr' if idjk==0 else None)
    
    ax.set_ylim([1.1,1.2])
    ax.legend(fontsize=10)

run(1,200,[1,2,4,8,16])
yu.finalizePlot('autocorrelation_toymodel')

In [3]:
def symmetrize(dat):
    dat_bw=np.roll(np.flip(dat,axis=1),1,axis=1)
    return((dat[:,:,0]+dat[:,:,5]) - (dat_bw[:,:,10]+dat_bw[:,:,15]))/4

Nsrc=100
path=f'/capstor/store/cscs/userlab/lp139/lyan/code/projectData/03_NpiScatteringWilson/MILC_a09m130W/data_merge/NpiScatteringWilson_Nsrc={Nsrc}.h5'
data={}
with h5py.File(path) as f:    
    cfgs_all=[cfg.decode() for cfg in f['cfgs'][:]]
    # cfgs_map=[cfgs_all.index(cfg) for cfg in cfgs]
    
    if True:
        cfgs_map=[cfgs_all.index(cfg) for cfg in cfgs_all]
    
    for cont in f['data'].keys():
        if cont in ['N']:
            moms=yu.moms2list(f['momsN'][:])
            t=f[f'data/{cont}'][:,:,moms.index([0,0,0])]
            t=symmetrize(t)
            data[cont]=t[cfgs_map]
            continue
        if cont in ['P']:
            t=f[f'data/{cont}'][:,:,0]
            t_bw=np.roll(np.flip(t,axis=1),1,axis=1)
            t=(t+t_bw)/2
            data[cont]=t[cfgs_map]
            continue
        t=f[f'data/{cont}'][:,:,0]
        t=symmetrize(t)
        data[cont]=t[cfgs_map]

In [ ]:
fig,axs=yu.getFigAxs(1,1,Lrow=4,Lcol=12)
ax=axs[0,0]
xunit=1; yunit=1
# ax.set_ylim(52,56)

ax.set_ylim(-1,2)

djks=[1,2,4,8,16]

dat=np.real(data['P'])
Ncfg_old=len(dat)
Ncfg_new=djks[-1]*(Ncfg_old//djks[-1])
dat=dat[:Ncfg_new]
print(f'{Ncfg_old}->{Ncfg_new}={djks[-1]}*{Ncfg_old//djks[-1]}')
ts=np.arange(1,yun.ens2NT[ens]//2,3)

mean,err,tauint,dt=yu.tauint(dat)
print(ts)
print(np.array(yu.un2str(tauint*2,dt*2))[ts])

plt_x=(ts-0.3)*xunit; plt_y=mean[ts]*yunit*0; plt_yerr=err[ts]*yunit
ax.errorbar(plt_x,plt_y,plt_yerr,color='black',label=f'puwr')

ts_title=ts[0:-1:4]
ts_title_str=','.join([str(ele) for ele in ts_title])
tau_int_str=','.join(np.array(yu.un2str(tauint*2,dt*2))[ts_title])
fig.suptitle(f'ts={ts_title_str}; 2*tau_int={tau_int_str}')

for idjk,djk in enumerate(djks):
    dat_jk=yu.jackknife(dat,d=djk)
    mean,err=yu.jackme(dat_jk)

    plt_x=(ts+idjk*0.3)*xunit; plt_y=mean[ts]*yunit*0; plt_yerr=err[ts]*yunit
    ax.errorbar(plt_x,plt_y,plt_yerr,color=yu.colors8[idjk],label=f'{djk}')
ax.legend(ncol=2)
yu.finalizePlot('c2pt_pion')

339->336=16*21
[ 1  4  7 10 13 16 19 22 25 28 31 34 37 40 43 46]
['2.31(46)' '2.34(53)' '2.34(53)' '2.33(53)' '2.33(53)' '2.31(52)'
 '2.33(53)' '2.35(53)' '2.37(54)' '2.39(54)' '2.40(54)' '2.42(55)'
 '2.40(54)' '2.39(54)' '2.40(54)' '2.40(54)']


In [103]:
dat=np.real(data['P'])
mean,err=yu.jackknife2_autocorrelation(dat)

fig,axs=yu.getFigAxs(1,1,Lrow=6,Lcol=12)
ax=axs[0,0]
ax.set_xlabel('conf')
ax.set_ylabel(r'$\Gamma(\tau)$')
ax.set_xlim([0,len(mean)])
ax.set_xlim([0,9])
ax.set_ylim([-1,1])
ax.set_yticks(np.arange(-1,1.1,0.25))
yu.addRefLine(ax,0)
xunit=1; yunit=1
ts=[4,8,16]
# ts=[10,20,30]

for i,t in enumerate(ts):
    taus=np.arange(len(mean))
    plt_x=taus*xunit; plt_y=mean[taus,t]*yunit; plt_yerr=err[taus,t]*yunit
    ax.plot(plt_x,plt_y,color=yu.colors8[i],ls='--',marker='',label=f't/a={t}')
    ax.fill_between(plt_x,plt_y-plt_yerr,plt_y+plt_yerr,color=yu.colors8[i],alpha=0.2)
    
ax.legend()
ax.grid()
yu.finalizePlot('Gamma_tau_c2pt_pion')

In [43]:
fig,axs=yu.getFigAxs(1,1,Lrow=4,Lcol=12)
ax=axs[0,0]
ax.set_ylim([138,145])

xunit=1; yunit=yun.ens2aInv[ens]

# ax.set_ylim([140.67-0.4,140.67+0.4])
# yu.addRefLine(ax,140.67-0.28)
# yu.addRefLine(ax,140.67-0.37)
# yu.addRefLine(ax,140.67-0.28+0.09)
# yu.addRefLine(ax,140.67-0.28+0.09*2)
# yu.addRefLine(ax,140.67-0.28+0.09*3)

djks=[1,2,4,8,16]

dat=np.real(data['P'])
Ncfg_old=len(dat)
Ncfg_new=djks[-1]*(Ncfg_old//djks[-1])
dat=dat[:Ncfg_new]
print(f'{Ncfg_old}->{Ncfg_new}={djks[-1]}*{Ncfg_old//djks[-1]}')
nshow_min=2; nshow_max=6
ts=np.arange(1,yun.ens2NT[ens]//2,3)
dat=dat[:,:max(ts)+4]

c2pt2meff_pion=lambda c2pt:yu.c2pt2meff_pion(c2pt,yun.ens2NT[ens])

mean,err,tauint,dt=yu.tauint(dat,c2pt2meff_pion)
print(ts)
print(np.array(yu.un2str(tauint*2,dt*2))[ts])

plt_x=(ts-0.3)*xunit; plt_y=mean[ts]*yunit; plt_yerr=err[ts]*yunit
ax.errorbar(plt_x,plt_y,plt_yerr,color='black',label=f'puwr')

ts_title=ts[0:-1:4]
ts_title_str=','.join([str(ele) for ele in ts_title])
tau_int_str=','.join(np.array(yu.un2str(tauint*2,dt*2))[ts_title])
fig.suptitle(f'ts={ts_title_str}; 2*tau_int={tau_int_str}')

for idjk,djk in enumerate(djks):
    dat_jk=yu.jackknife(dat,d=djk)
    meff=yu.jackmap(c2pt2meff_pion,dat_jk)
    mean,err=yu.jackme(meff)
    
    plt_x=(ts+idjk*0.3)*xunit; plt_y=mean[ts]*yunit; plt_yerr=err[ts]*yunit
    ax.errorbar(plt_x,plt_y,plt_yerr,color=yu.colors8[idjk],label=f'{djk}')
    
ax.legend(ncol=2)
yu.finalizePlot('meff_pion')

339->336=16*21
[ 1  4  7 10 13 16 19 22 25 28 31 34 37 40 43 46]
['2.25(45)' '1.94(40)' '1.55(28)' '1.68(35)' '1.80(37)' '1.83(38)'
 '1.80(37)' '2.00(41)' '1.89(39)' '1.74(36)' '1.52(27)' '1.55(28)'
 '1.20(22)' '1.49(27)' '1.41(26)' '0.97(15)']


In [44]:
fig,axs=yu.getFigAxs(1,1,Lrow=4,Lcol=12)
ax=axs[0,0]
ax.set_ylim([930,980])

xunit=yun.ens2a[ens]; yunit=yun.ens2aInv[ens]
xunit=1

djks=[1,2,4,8,16]

dat=np.real(data['N'])
Ncfg_old=len(dat)
Ncfg_new=djks[-1]*(Ncfg_old//djks[-1])
dat=dat[:Ncfg_new]
print(f'{Ncfg_old}->{Ncfg_new}={djks[-1]}*{Ncfg_old//djks[-1]}')
nshow_min=10; nshow_max=14
ts=np.arange(1,24)
dat=dat[:,:max(ts)+4]

def func(c2pt):
    t=yu.c2pt2meff(c2pt)
    t=np.array([ele if not np.isnan(ele) else 0 for ele in t])
    return t

mean,err,tauint,dt=yu.tauint(dat,func)
print(ts)
print(np.array(yu.un2str(tauint*2,dt*2))[ts])

plt_x=(ts-0.1)*xunit; plt_y=mean[ts]*yunit; plt_yerr=err[ts]*yunit
ax.errorbar(plt_x,plt_y,plt_yerr,color='black',label=f'puwr')

ts_title=ts[0:30:6]
ts_title_str=','.join([str(ele) for ele in ts_title])
tau_int_str=','.join(np.array(yu.un2str(tauint*2,dt*2))[ts_title])
fig.suptitle(f'ts={ts_title_str}; 2*tau_int={tau_int_str}')

for idjk,djk in enumerate(djks):
    dat_jk=yu.jackknife(dat,d=djk)
    meff=yu.jackmap(yu.c2pt2meff,dat_jk)
    mean,err=yu.jackme(meff)

    plt_x=(ts+idjk*0.1)*xunit; plt_y=mean[ts]*yunit; plt_yerr=err[ts]*yunit
    ax.errorbar(plt_x,plt_y,plt_yerr,color=yu.colors8[idjk],label=f'{djk}')
    
ax.legend()
yu.finalizePlot('meff_N')

339->336=16*21
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
['1.50(27)' '1.11(21)' '1.06(20)' '1.26(23)' '0.98(15)' '0.93(14)'
 '1.28(24)' '1.24(23)' '1.20(22)' '0.94(15)' '0.90(14)' '1.12(21)'
 '1.23(23)' '0.97(15)' '0.93(14)' '1.14(21)' '0.84(13)' '0.91(14)'
 '1.30(24)' '1.18(22)' '1.21(22)' '0.91(14)' '0.92(18)']
